In [5]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [6]:
!pip uninstall jaxlib
#!pip install jaxlib==0.4.2
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.3.25+cuda11.cudnn805.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.2%2Bcuda11.cudnn82-cp38-cp38-manylinux2014_x86_64.whl (164.4 MB)


In [1]:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
#%env JAX_PLATFORMS=cpu
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [4]:
from utils import *

In [5]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [6]:
from typing import Callable, Iterable, Optional, Sequence, Tuple, Union
from neural_tangents import Kernel
from neural_tangents._src.stax.requirements import Bool, Diagonal, get_diagonal_outer_prods, layer, mean_and_var, requires, supports_masking
from jax import numpy as np
import jax

# pattern is a two dimensional array of shape k x 4
# a edge list from the 4 dimensional intersected adjacency matrix
# R = np.transpose(np.array(np.nonzero(As_int)))
# As_int is four dimensional with batch x #nodes x #nodes x #nodes
# e.g. As_int[a,b,c,:] gives all nodes, which are neighboors
# of nodes b and c in batch a.

# Thus e.g a row from the pattern array [A,B,C,D] tells us,
# that node D is a neighboor of nodes B and C in batch A.

@layer
@supports_masking(remask_kernel=False)
def two_wl_aggregation(n_nodes):
    """
    Return a layer, that implements the gatter
    and scatter operations given the reference Matrix 
    R.
    R is of shape (#graps in the batch *
    # edges of each grap) x 2
    """
    
    init_fn = lambda rng, input_shape: (input_shape, ())
    
    def apply_fn(params,
               inputs: np.ndarray,
               *,
               pattern: Optional[np.ndarray] = None,
               **kwargs):
        num_segments = inputs.shape[0] * n_nodes**2
        
        # edges from v_i to v_j
        # pattern := [A,B,C,D]
        # A = batch, B = node i, C = node j
        # linear_index([A,B,C]) => if we have a batched adjacency matrix
        # what is the index of nodes B, C in batch A, if we index the 
        # array 3 dimensional array linear (c-style)
        e_ij = linear_index(pattern[:,[0,1,2]], inputs.shape)
        # edges from v_i to v_l
        e_il = linear_index(pattern[:,[0,1,3]], inputs.shape)
        # edges from v_l to v_j
        e_lj = linear_index(pattern[:,[0,3,2]], inputs.shape)

        graphs_edge_features = np.reshape(inputs, (-1, inputs.shape[3]))
        x_gamma_1 = np.take(graphs_edge_features, e_il, axis=0)
        x_gamma_2 = np.take(graphs_edge_features, e_lj, axis=0)
        X_gamma_1_sum = jax.ops.segment_sum(x_gamma_1, e_ij, num_segments)
        X_gamma_2_sum = jax.ops.segment_sum(x_gamma_2, e_ij, num_segments)
        X_gamma_sum = np.append(np.expand_dims(X_gamma_1_sum, 1), np.expand_dims(X_gamma_2_sum, 1), 1)
        X_gamma_sum = np.sum(X_gamma_sum, 1)

        out = np.reshape(X_gamma_sum, inputs.shape)
        return out


    def kernel_fn(k: Kernel,
                *,
                pattern: Tuple[Optional[np.ndarray],
                               Optional[np.ndarray]] = (None, None),
                **kwargs):
        
        num_segments = int(np.prod(np.array(k.ntk.shape)))
        
        # arrange the incoming kernel matrix as a flatt array
        # ntk_linear = np.reshape(k.ntk, (-1, 1))
        # we dont need to reshape, as take works also on the multidimensional array

        # a double sum is one sum over the karthesian product of 
        # the sets the two sums sum.
        # pattern has columns: batch, node i, node j, node a
        # the karthesian product (A x B) ist than: 
        # b_A, i_A, j_A, a_A, b_B, i_B, j_B, a_B
        #   0,   1,   2,   3,   4,   5,   6,   7
        # rearrange the columns to: 
        # b_A, b_B, i_A, i_B, j_A, j_B, a_A, a_B
        #   0,   1,   2,   3,   4,   5,   6,   7
        patterns = row_wise_karthesian_prod(pattern[0], pattern[1])       
        patterns = patterns[:,[0,4,1,5,2,6,3,7]]
        # "Interpretation" of the kartesian product:
        # A row from the pattern array [A,B,C,D] tells us,
        # that node D is a neighboor of nodes B and C in batch A.
        # Lets consider only the rows, where columns A,B,C are equal (node e_ij)
        # column D is then the "set" of nodes the "neigborhood" aggregation
        # sum for e_ij sums.
        # Also, consider the pathern from the other graph with 
        # columns [Ab,Bb,Cb,Db]. When take only the rows where 
        # columns Ab, Bb, Cb are equal. Column Db makes then the 
        # indexes to sum over for the "neigborhood" aggregation.
        # To create the double sum we need the 
        # karthesian product of the indexes in column D and 
        # column Db.

        # note: ib <=> i bar
        e_i_j_ib_jb = linear_index(patterns[:,[0,1,2,4,3,5]], k.ntk.shape)
        e_i_a_ib_ab = linear_index(patterns[:,[0,1,2,6,3,7]], k.ntk.shape)
        e_i_a_ab_jb = linear_index(patterns[:,[0,1,2,6,7,5]], k.ntk.shape)
        e_a_j_ib_ab = linear_index(patterns[:,[0,1,6,4,3,7]], k.ntk.shape)
        e_a_j_ab_jb = linear_index(patterns[:,[0,1,6,4,7,5]], k.ntk.shape)

        theta_i_a_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ib_ab), e_i_j_ib_jb, num_segments)
        theta_i_a_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ab_jb), e_i_j_ib_jb, num_segments)
        theta_a_j_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ib_ab), e_i_j_ib_jb, num_segments)
        theta_a_j_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ab_jb), e_i_j_ib_jb, num_segments)

        thetas_linear = np.zeros((theta_i_a_ab_jb.shape[0],1))
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ib_ab, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ab_jb, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ib_ab, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ab_jb, 1), 1)

        theta_linear = np.sum(thetas_linear, 1)
        theta = np.reshape(theta_linear, k.ntk.shape)
        thera = theta + k.ntk
        
        return k.replace(ntk=theta,
                     is_gaussian=True,
                     is_input=False)
    
    return init_fn, apply_fn, kernel_fn

In [7]:
#from layers import two_wl_aggregation
# for input "kernel_fn(x1, x2, 'ntk', pattern = (p1,p2))"
# the shape of the kernel after the convolution is:
# x1 = (b1, h1, w1, c1)
# x2 = (b2, h2, w2, c2)
# k = (b1, b2, h1, h2, w1, w2) 


L_branche = stax.serial(
    stax.Conv(100, (1,1)),
    )

Gamma_branche = stax.serial(
    stax.Conv(100, (1,1)),
    two_wl_aggregation(n_nodes=28),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
    )


init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer, # k = (b1, b2, h1, h2, w1, w2)
    stax.GlobalSumPool(), # # k = (b1, b2)
)




In [8]:
# this needs ~9GB gpu-ram, takes ~15s on GPU ~2min on CPU (size 4 using XLA_PYTHON_CLIENT_PREALLOCATE=false)
%%time 
size = 20
r = 9
i = 0
j = 0

path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}"

x1 = np.load(path + f"/x1_{i*size}_{(1+i)*size}.npy")
x2 = np.load(path + f"/x2_{j*size}_{(1+j)*size}.npy")
p1 = np.load(path + f"/p1_{i*size}_{(1+i)*size}.npy")
p2 = np.load(path + f"/p2_{j*size}_{(1+j)*size}.npy")

kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))

XlaRuntimeError: ignored

In [22]:
kernel_matrix.shape

(10, 10)

In [8]:
18**2 * 15 / 3600

1.35

In [9]:
import shutil
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/")

In [10]:
import os
newpath = "/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [8]:
# at some point this allways fails with 15gb gpu memory
# fails always for some function in utils
# can we make those function more memory efficient?

# with XLA_PYTHON_CLIENT_ALLOCATOR=platform this seems to stay below 5GB 
# GPU-RAM usage for size=4

# calculate kernel from saved preprocessed data
size = 10
r = 18
path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}" 

for i in range(1,r):
  print(i)
  for j in range(r):

    x1 = np.load(path + f"/x1_{i*size}_{(1+i)*size}.npy")
    x2 = np.load(path + f"/x2_{j*size}_{(1+j)*size}.npy")
    p1 = np.load(path + f"/p1_{i*size}_{(1+i)*size}.npy")
    p2 = np.load(path + f"/p2_{j*size}_{(1+j)*size}.npy")

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/kernel_matrix_{(1+i)*size}_{(1+j)*size}", kernel_matrix)

1


/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
